更改bg文件命名

In [6]:
import os
import re
import logging

def get_natural_sort_key(string):
    """ Generates a key for natural sorting of strings containing numbers. """
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', string)]

def rename_files_in_directory(directory, category, suffix):
    """ Renames all files in a specified directory following a natural sort order. """
    try:
        filenames = sorted([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))], key=get_natural_sort_key)
        for index, filename in enumerate(filenames, start=1):
            old_file = os.path.join(directory, filename)
            new_filename = f"{index}.{suffix}"
            new_file = os.path.join(directory, new_filename)
            os.rename(old_file, new_file)
            logging.info(f"Renamed '{filename}' to '{new_filename}'")
    except Exception as e:
        logging.error(f"Error renaming files in {directory}: {e}")

def batch_rename_files_in_nested_dirs(base_directory, suffix):
    """ Recursively renames files in all nested directories within the base directory. """
    try:
        for category in os.listdir(base_directory):
            category_path = os.path.join(base_directory, category)
            for image_folder_name in os.listdir(category_path):
                image_folder_dir = os.path.join(category_path, image_folder_name)
                if os.path.isdir(image_folder_dir):
                    rename_files_in_directory(image_folder_dir, category, suffix)
    except Exception as e:
        logging.error(f"Error processing directory {base_directory}: {e}")

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    suffix = 'jpg'
    base_directory = './background/train/image'
    batch_rename_files_in_nested_dirs(base_directory, suffix)


INFO:root:Renamed '1.jpg' to '1.jpg'
INFO:root:Renamed '2.jpg' to '2.jpg'
INFO:root:Renamed '3.jpg' to '3.jpg'
INFO:root:Renamed '4.jpg' to '4.jpg'
INFO:root:Renamed '5.jpg' to '5.jpg'
INFO:root:Renamed '6.jpg' to '6.jpg'
INFO:root:Renamed '7.jpg' to '7.jpg'
ERROR:root:Error renaming files in ./background/train/image\ant\ant_1: [WinError 183] 当文件已存在时，无法创建该文件。: './background/train/image\\ant\\ant_1\\8.jpeg' -> './background/train/image\\ant\\ant_1\\8.jpg'
INFO:root:Renamed '1.jpg' to '1.jpg'
INFO:root:Renamed '2.jpg' to '2.jpg'
INFO:root:Renamed '3.jpg' to '3.jpg'
INFO:root:Renamed '4.jpg' to '4.jpg'
INFO:root:Renamed '5.jpg' to '5.jpg'
INFO:root:Renamed '6.jpg' to '6.jpg'
INFO:root:Renamed '7.jpg' to '7.jpg'
INFO:root:Renamed '8.jpg' to '8.jpg'
INFO:root:Renamed '9.jpg' to '9.jpg'
INFO:root:Renamed '10.jpg' to '10.jpg'
INFO:root:Renamed '1.jpg' to '1.jpg'
INFO:root:Renamed '2.jpg' to '2.jpg'
INFO:root:Renamed '3.jpg' to '3.jpg'
INFO:root:Renamed '4.jpg' to '4.jpg'
INFO:root:Renamed '5.

更改原图文件命名

In [2]:
import os
import re

def natural_sort_key(s):
    """Helper function to turn a string into a list of strings and digits for natural sorting"""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

def batch_rename_files_sorted(directory, category, suffix):
    # Retrieve all file names and sort them using natural sort
    filenames = sorted([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))], key=natural_sort_key)

    # Rename files in the format 'category_n.jpg'
    for index, filename in enumerate(filenames, start=1):
        old_file = os.path.join(directory, filename)
        new_filename = f"{index}.{suffix}"  # Format the new filename
        new_file = os.path.join(directory, new_filename)

        # Rename the file
        os.rename(old_file, new_file)
        print(f"Renamed '{filename}' to '{new_filename}'")

# Usage example
category = 'ant'
suffix = 'jpg'

directory_path = os.path.join('./background/train/image/', category)
batch_rename_files_sorted(directory_path, category, suffix)


图片转换：

In [20]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

def get_sorted_folders(directory):
    """返回指定目录下所有文件夹名称的排序列表"""
    folders = [item for item in os.listdir(directory) if os.path.isdir(os.path.join(directory, item))]
    return sorted(folders)

def inpaint_background(image_path, mask_path, save_path):
    """Inpaint the object area in the image using the mask and save the result if the save path is not already occupied."""
    # Check if the output file already exists
    if not os.path.exists(save_path):
        # Load the image and mask
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Make sure the mask has the same size as the image
        if image.shape[:2] != mask.shape[:2]:
            mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        # Inpaint the areas where mask is white (object areas in the mask)
        inpaint_radius = 3  # Radius of a circular neighborhood of each point inpainted
        inpainted_image = cv2.inpaint(image, mask, inpaint_radius, cv2.INPAINT_TELEA)
        
        # Save the inpainted image
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        cv2.imwrite(save_path, inpainted_image)


def process_images(base_dir):
    """处理指定基础目录下的所有图片"""
    image_dir = os.path.join(base_dir, 'train', 'image')
    mask_dir = os.path.join(base_dir, 'train', 'groundtruth')
    save_dir = './background_transformed/train/image'

    categories = get_sorted_folders(image_dir)

    for category in categories:
        category_image_dir = os.path.join(image_dir, category)
        category_mask_dir = os.path.join(mask_dir, category)
        category_save_dir = os.path.join(save_dir, category)

        filenames = os.listdir(category_image_dir)
        for filename in tqdm(filenames, desc=f"Processing {category}"):
            if filename.endswith('.jpg'):
                image_path = os.path.join(category_image_dir, filename)
                mask_filename = filename.replace('.jpg', '.png')
                mask_path = os.path.join(category_mask_dir, mask_filename)
                save_path = os.path.join(category_save_dir, filename)

                inpaint_background(image_path, mask_path, save_path)

# 指定基础目录
base_dir = './CoCOD8K'

# 处理基础目录下的所有图片
process_images(base_dir)


Processing bird:   0%|          | 0/265 [00:00<?, ?it/s]

Processing worm: 100%|██████████| 159/159 [00:11<00:00, 14.05it/s]


爬取图片

In [5]:
import json
import os
import requests
import time
import re
import logging
import concurrent.futures
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.options import Options

# Constants and Configurations
BASE_DIR = './background_transformed/train/image/'
OUTPUT_DIR = './background/train/image/'
MAX_NUM_IMAGES = 15
WAIT_TIME = 10
SCROLL_PAUSE_TIME = 1  # Reduced pause time for faster scrolling
IMAGE_CACHE = set()  # Cache to store downloaded image URLs
MAX_THREADS = 16  # Maximum number of threads for ThreadPoolExecutor

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def numerical_sort_key(s):
    """Sort function for natural order sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def download_image(url, folder_path, image_number):
    """Download an image from a URL and save it to a specified folder."""
    # Ensure the output folder exists
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    image_file_path = os.path.join(folder_path, f"{image_number}.jpg")
    if os.path.exists(image_file_path):
        return

    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(image_file_path, 'wb') as file:
            file.write(response.content)
        IMAGE_CACHE.add(url)  # Add URL to cache after successful download
    except Exception as e:
        logging.error(f"Failed to download {url}. Error: {e}")


def download_images_threaded(image_urls, output_folder):
    """Download images using ThreadPoolExecutor."""
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = [executor.submit(download_image, url, output_folder, i+1)
                   for i, url in enumerate(image_urls) if url not in IMAGE_CACHE]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                logging.error(f"Failed to download image: {e}")

def scroll_to_load_images(driver):
    """Scroll through the webpage to load all images with optimized scrolling."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down a bit more each time to load more content
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight / 2);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def search_similar_images(image_path, driver, output_folder):
    """Search for similar images using Bing Visual Search and download them using threading."""
    try:
        driver.get('https://www.bing.com/visualsearch')
        upload_button = WebDriverWait(driver, WAIT_TIME).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'pstpn'))
        )
        upload_button.click()

        file_input = WebDriverWait(driver, WAIT_TIME).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='file']"))
        )
        file_input.send_keys(os.path.abspath(image_path))

        WebDriverWait(driver, WAIT_TIME).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.richImgLnk'))
        )

        scroll_to_load_images(driver)

        image_links = driver.find_elements(By.CSS_SELECTOR, 'a.richImgLnk')
        image_urls = []
        for link in image_links[:MAX_NUM_IMAGES]:
            data_m = link.get_attribute('data-m')
            if data_m:
                data = json.loads(data_m)
                img_url = data.get('murl')
                if img_url:
                    image_urls.append(img_url)
        
        download_images_threaded(image_urls, output_folder)  # Call to threaded download function
    except Exception as e:
        logging.error(f"An error occurred while searching images: {e}")

def process_categories(base_dir):
    """Process each category and image in the base directory with headless browser."""
    # Configure headless browser
    options = Options()
    options.headless = True  # Enable headless mode
    driver = webdriver.Edge()
    
    try:
        categories = sorted(os.listdir(base_dir), key=numerical_sort_key)
        for category in tqdm(categories, desc="Processing categories"):
            category_path = os.path.join(base_dir, category)
            if os.path.isdir(category_path):
                images = sorted(os.listdir(category_path), key=numerical_sort_key)
                for img_name in tqdm(images, desc=f"Processing images in {category}"):
                    img_path = os.path.join(category_path, img_name)
                    output_folder = os.path.join(OUTPUT_DIR, img_name.split('_')[0], img_name.split('.')[0])
                    if not os.path.exists(output_folder):
                        search_similar_images(img_path, driver, output_folder)
    finally:
        driver.quit()

if __name__ == '__main__':
    process_categories(BASE_DIR)

2024-01-29 10:14:08,796 - WARNING - The msedgedriver version (120.0.2210.144) detected in PATH at c:\Users\ms i\AppData\Local\Programs\Python\Python312\msedgedriver.exe might not be compatible with the detected MicrosoftEdge version (121.0.2277.83); currently, msedgedriver 121.0.2277.83 is recommended for MicrosoftEdge 121.*, so it is advised to delete the driver in PATH and retry
Processing categories:  76%|███████▌  | 53/70 [00:15<00:00, 67.47it/s]2024-01-29 10:14:29,626 - ERROR - An error occurred while searching images: Message: 

Processing categories: 100%|██████████| 70/70 [00:19<00:00,  3.53it/s]
